### Обзор 2
Этот ноутбук посвящён построению **baseline-модели** для прогнозирования цен на жильё.  
Используются предварительно обработанные данные из ноутбука `Sales_House_EDA.ipynb`.


In [2]:
#быстрый baseline на данных df_baseline с использованием KNN,
#LinearRegression и DecisionTree, с масштабированием числовых признаков и оценкой на тестовой выборке.
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "4"  # или сколько у вас ядер

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# 1) Загружаем данные

df = pd.read_csv("data/Preprocessed_Data.csv")


# 2) Определяем признаки и таргет

# Берём все колонки кроме 'target' и 'target_log' как признаки
target_col = 'target'
X = df.drop(columns=['target', 'target_log'])
y = df[target_col]


# 3) Разделение на тренировку и тест

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# 4) Масштабирование числовых признаков

# KNN чувствителен к масштабу
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# 5) Список базовых моделей

models = {
    "LinearRegression": LinearRegression(),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "KNN": KNeighborsRegressor(n_neighbors=5)
}


# 6) Обучение и оценка моделей

results = []

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": name,
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "R2": r2
    })

baseline_df = pd.DataFrame(results)
print(baseline_df)



              Model           MSE          RMSE            MAE        R2
0  LinearRegression  2.379320e+12  1.542505e+06  517501.914515  0.228672
1      DecisionTree  2.085670e+12  1.444185e+06  241523.377025  0.323868
2               KNN  1.408487e+12  1.186797e+06  269855.410695  0.543397


In [4]:
#Быстрый перебор KNN с разными n_neighbors

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# 1) Загружаем данные

df = pd.read_csv("data/Preprocessed_Data.csv")


# 2) Определяем таргет и признаки

target_col = 'target_log'  # логарифм цены
X = df.drop(columns=['target', 'target_log'])
y = df[target_col]


# 3) Новые числовые признаки для масштабирования

original_numerical = ['sqft', 'beds', 'baths', 'stories']
all_numerical = X.select_dtypes(include=np.number).columns.tolist()
new_numerical = [col for col in all_numerical if col not in original_numerical]


# 4) Разделение на тренировку и тест

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# 5) Масштабирование только новых числовых признаков

scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[new_numerical] = scaler.fit_transform(X_train[new_numerical])
X_test_scaled[new_numerical] = scaler.transform(X_test[new_numerical])


# 6) LinearRegression и DecisionTree

baseline_models = {
    "LinearRegression": LinearRegression(),
    "DecisionTree": DecisionTreeRegressor(random_state=42)
}

results = []

for name, model in baseline_models.items():
    model.fit(X_train_scaled, y_train)
    y_pred_log = model.predict(X_test_scaled)
    
    # Реальные цены
    y_test_real = np.exp(y_test)
    y_pred_real = np.exp(y_pred_log)
    
    results.append({
        "Model": name,
        "RMSE_log": np.sqrt(mean_squared_error(y_test, y_pred_log)),
        "R2_log": r2_score(y_test, y_pred_log),
        "MAE_real": mean_absolute_error(y_test_real, y_pred_real),
        "RMSE_real": np.sqrt(mean_squared_error(y_test_real, y_pred_real)),
        "R2_real": r2_score(y_test_real, y_pred_real)
    })


# 7) Быстрый перебор KNN для k=1..20

best_knn_r2 = -np.inf
best_knn_model = None
best_k = None

for k in range(1, 21):
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    y_pred_log = knn.predict(X_test_scaled)
    r2 = r2_score(y_test, y_pred_log)
    
    if r2 > best_knn_r2:
        best_knn_r2 = r2
        best_knn_model = knn
        best_k = k

# Метрики для лучшего KNN
y_pred_log = best_knn_model.predict(X_test_scaled)
y_pred_real = np.exp(y_pred_log)

results.append({
    "Model": f"KNN (k={best_k})",
    "RMSE_log": np.sqrt(mean_squared_error(y_test, y_pred_log)),
    "R2_log": r2_score(y_test, y_pred_log),
    "MAE_real": mean_absolute_error(y_test_real, y_pred_real),
    "RMSE_real": np.sqrt(mean_squared_error(y_test_real, y_pred_real)),
    "R2_real": r2_score(y_test_real, y_pred_real)
})


# 8) Таблица результатов

baseline_df = pd.DataFrame(results)
print(baseline_df)
print(f"\nЛучший KNN использует k = {best_k} с R² (log) = {best_knn_r2:.4f}")


              Model  RMSE_log    R2_log       MAE_real     RMSE_real   R2_real
0  LinearRegression  0.660293  0.667810  319954.233254  2.968816e+06 -1.857279
1      DecisionTree  0.562797  0.758667  233505.576198  1.478898e+06  0.290974
2         KNN (k=7)  0.538571  0.778997  251056.690831  1.206036e+06  0.528473

Лучший KNN использует k = 7 с R² (log) = 0.7790


На логарифме цен KNN показывает лучший R² → хорошо объясняет структуру цен.
На реальных ценах KNN также выигрывает по RMSE_real и R²_real, хотя MAE чуть выше, чем у DecisionTree.
Значение k=7 для KNN оптимально в данном эксперименте.

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor


# 1) Загружаем данные

df = pd.read_csv("data/Preprocessed_Data.csv")


# 2) Определяем таргет и признаки

target_col = 'target_log'  # логарифм цены
X = df.drop(columns=['target', 'target_log'])
y = df[target_col]


# 3) Новые числовые признаки для масштабирования

original_numerical = ['sqft', 'beds', 'baths', 'stories']
all_numerical = X.select_dtypes(include=np.number).columns.tolist()
new_numerical = [col for col in all_numerical if col not in original_numerical]

# 4) Разделение на трен. и тест.

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 5) Масштабирование только новых числовых признаков

scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[new_numerical] = scaler.fit_transform(X_train[new_numerical])
X_test_scaled[new_numerical] = scaler.transform(X_test[new_numerical])


# 6) LinearRegression и DecisionTree

baseline_models = {
    "LinearRegression": LinearRegression(),
    "DecisionTree": DecisionTreeRegressor(random_state=42)
}

results = []

for name, model in baseline_models.items():
    model.fit(X_train_scaled, y_train)
    y_pred_log = model.predict(X_test_scaled)
    
    # Реальные цены
    y_test_real = np.exp(y_test)
    y_pred_real = np.exp(y_pred_log)
    
    results.append({
        "Model": name,
        "RMSE_log": np.sqrt(mean_squared_error(y_test, y_pred_log)),
        "R2_log": r2_score(y_test, y_pred_log),
        "MAE_real": mean_absolute_error(y_test_real, y_pred_real),
        "RMSE_real": np.sqrt(mean_squared_error(y_test_real, y_pred_real)),
        "R2_real": r2_score(y_test_real, y_pred_real)
    })


# 7) Быстрый перебор KNN для k=1..20

best_knn_r2 = -np.inf
best_knn_model = None
best_k = None

for k in range(1, 21):
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    y_pred_log = knn.predict(X_test_scaled)
    r2 = r2_score(y_test, y_pred_log)
    
    if r2 > best_knn_r2:
        best_knn_r2 = r2
        best_knn_model = knn
        best_k = k

# Метрики для лучшего KNN
y_pred_log = best_knn_model.predict(X_test_scaled)
y_pred_real = np.exp(y_pred_log)

results.append({
    "Model": f"KNN (k={best_k})",
    "RMSE_log": np.sqrt(mean_squared_error(y_test, y_pred_log)),
    "R2_log": r2_score(y_test, y_pred_log),
    "MAE_real": mean_absolute_error(y_test_real, y_pred_real),
    "RMSE_real": np.sqrt(mean_squared_error(y_test_real, y_pred_real)),
    "R2_real": r2_score(y_test_real, y_pred_real)
})


# 8) Таблица результатов

baseline_df = pd.DataFrame(results)
print(baseline_df)
print(f"\nЛучший KNN использует k = {best_k} с R² (log) = {best_knn_r2:.4f}")


              Model  RMSE_log    R2_log       MAE_real     RMSE_real   R2_real
0  LinearRegression  0.660293  0.667810  319954.233254  2.968816e+06 -1.857279
1      DecisionTree  0.562797  0.758667  233505.576198  1.478898e+06  0.290974
2         KNN (k=7)  0.538571  0.778997  251056.690831  1.206036e+06  0.528473

Лучший KNN использует k = 7 с R² (log) = 0.7790
